In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：为批量预测训练自动ML文本实体提取模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_text_entity_extraction_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_text_entity_extraction_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_automl_text_entity_extraction_batch.ipynb">
      在Google Cloud笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述


本教程演示了如何使用Vertex SDK创建文本实体提取模型，并使用Google Cloud的自动机器学习模型进行批量预测。您可以在[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)了解更多信息。

数据集

本教程使用的数据集是来自国家生物技术信息中心的[NCBI疾病研究摘要数据集](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/)。您在本教程中将使用的数据集版本存储在公共云存储桶中。

### 目标

在本教程中，您将从Python脚本中创建一个AutoML文本实体提取模型，然后使用Vertex SDK进行批量预测。您还可以选择使用`gcloud`命令行工具或在Cloud Console上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和使用在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，后台存储结果，并在准备好时将结果存储在Cloud Storage存储桶中。

费用

本教程使用谷歌云的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 的价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 的价格](https://cloud.google.com/storage/pricing)，并使用[价格计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成费用估算。

### 设置本地开发环境

如果您使用的是Colab或Google Cloud笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行Jupyter笔记本

[设置Python开发环境](https://cloud.google.com/python/setup)中的Cloud存储指南以及[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了一套简化的指令：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端命令行上运行`pip3 install jupyter`。

5. 要启动Jupyter，在终端命令行上运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装最新版本的Python用于Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新版本的 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

一旦您安装了额外的包，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU运行时

本教程不需要GPU运行时。

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。您第一次创建帐户时，您将获得$300的免费信用额度用于计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs、Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter会将以`!`开头的行作为shell命令运行，并且会对以`$`开头的Python变量进行插值。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改用于此笔记本其余部分操作的`REGION`变量。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶来训练Vertex AI。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享的测试帐户或项目。为避免在创建的资源上发生名称冲突，您应为每个实例会话创建一个时间戳，并将时间戳附加到您在此教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云帐户

**如果您正在使用谷歌云笔记本**，您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，按照提示进行身份验证。

**否则**，请按照以下步骤进行操作：

在云控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

在**将此服务帐号授权给项目**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，并选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您服务帐号密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论您的笔记本环境如何，以下步骤都是必需的。**

当您初始化用于Python的Vertex SDK时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源相关数据在不同会话之间保留的地方。

请在下方设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时才能运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

设置变量

接下来，设置一些在教程中使用的变量。
导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化用于Python的Vertex SDK

为您的项目和相应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

# 教程

现在你已经准备好开始创建自己的AutoML文本实体提取模型。

#### 云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为在云存储中JSONL索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/language/ucaip_ten_dataset.jsonl"

迅速查看您的数据

本教程使用存储在公共云存储桶中的NCBI生物医学数据集版本，使用一个JSONL索引文件。

首先快速查看数据。您可以通过计算JSONL索引文件中对象的数量（`wc -l`）来计算示例的数量，然后可查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，需要提供以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：要导入数据项到`Dataset`资源中的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.TextDataset.create(
    display_name="NCBI Biomedical" + "_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.text.extraction,
)

print(dataset.resource_name)

### 创建和运行训练管道

为了训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行这个管道。

#### 创建训练管道

使用`AutoMLTextTrainingJob`类创建一个AutoML训练管道，需要设置以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体抽取模型。
- `multi_label`：如果是一个分类任务，是单标签（False）还是多标签（True）。
- `sentiment_max`：如果是情感分析任务，情感值的最大值。

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLTextTrainingJob(
    display_name="biomedical_" + TIMESTAMP, prediction_type="extraction"
)

print(dag)

#### 运行训练流水线

接下来，通过调用 `run` 方法并传入以下参数来运行 DAG 以启动训练作业：

- `dataset`：用于训练模型的 `Dataset` 资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集比例。
- `test_fraction_split`：用于测试（留存数据）的数据集比例。
- `validation_fraction_split`：用于验证的数据集比例。

完成 `run` 方法后会返回 `Model` 资源。

训练流水线的执行可能需要最多 20 分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="biomedical_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 回顧模型評估分數
在您的模型訓練完成後，您可以查看其評估分數。

首先，您需要取得新模型的參考。與數據集一樣，您可以使用在部署模型時創建的模型變量的參考，或者列出項目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=biomedical_" + TIMESTAMP)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

发送一个批量预测请求

将一个批量预测发送到您部署的模型。

生成测试项目

您将使用合成数据作为测试数据项目。不必担心我们使用的是合成数据 - 我们只是想展示如何进行预测。

In [ ]:
test_item_1 = 'Molecular basis of hexosaminidase A deficiency and pseudodeficiency in the Berks County Pennsylvania Dutch.\tFollowing the birth of two infants with Tay-Sachs disease ( TSD ) , a non-Jewish , Pennsylvania Dutch kindred was screened for TSD carriers using the biochemical assay . A high frequency of individuals who appeared to be TSD heterozygotes was detected ( Kelly et al . , 1975 ) . Clinical and biochemical evidence suggested that the increased carrier frequency was due to at least two altered alleles for the hexosaminidase A alpha-subunit . We now report two mutant alleles in this Pennsylvania Dutch kindred , and one polymorphism . One allele , reported originally in a French TSD patient ( Akli et al . , 1991 ) , is a GT-- > AT transition at the donor splice-site of intron 9 . The second , a C-- > T transition at nucleotide 739 ( Arg247Trp ) , has been shown by Triggs-Raine et al . ( 1992 ) to be a clinically benign " pseudodeficient " allele associated with reduced enzyme activity against artificial substrate . Finally , a polymorphism [ G-- > A ( 759 ) ] , which leaves valine at codon 253 unchanged , is described'
test_item_2 = "Analysis of alkaptonuria (AKU) mutations and polymorphisms reveals that the CCC sequence motif is a mutational hot spot in the homogentisate 1,2 dioxygenase gene (HGO).	We recently showed that alkaptonuria ( AKU ) is caused by loss-of-function mutations in the homogentisate 1 , 2 dioxygenase gene ( HGO ) . Herein we describe haplotype and mutational analyses of HGO in seven new AKU pedigrees . These analyses identified two novel single-nucleotide polymorphisms ( INV4 + 31A-- > G and INV11 + 18A-- > G ) and six novel AKU mutations ( INV1-1G-- > A , W60G , Y62C , A122D , P230T , and D291E ) , which further illustrates the remarkable allelic heterogeneity found in AKU . Reexamination of all 29 mutations and polymorphisms thus far described in HGO shows that these nucleotide changes are not randomly distributed ; the CCC sequence motif and its inverted complement , GGG , are preferentially mutated . These analyses also demonstrated that the nucleotide substitutions in HGO do not involve CpG dinucleotides , which illustrates important differences between HGO and other genes for the occurrence of mutation at specific short-sequence motifs . Because the CCC sequence motifs comprise a significant proportion ( 34 . 5 % ) of all mutated bases that have been observed in HGO , we conclude that the CCC triplet is a mutational hot spot in HGO ."

制作批量输入文件

现在制作一个批量输入文件，您将将其存储在本地云存储桶中。批处理输入文件只能是 JSONL 格式。对于 JSONL 文件，您每行为每个数据项（实例）制作一个字典条目。该字典包含键/值对：

- `content`：文本项文件的云存储路径。
- `mime_type`：内容类型。在我们的示例中，它是一个`text`文件。

例如：

{'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

In [ ]:
import json

import tensorflow as tf

gcs_test_item_1 = BUCKET_NAME + "/test1.txt"
with tf.io.gfile.GFile(gcs_test_item_1, "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_NAME + "/test2.txt"
with tf.io.gfile.GFile(gcs_test_item_2, "w") as f:
    f.write(test_item_2 + "\n")

gcs_input_uri = BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用 batch_predict() 方法进行批量预测，需要传入以下参数：

- `job_display_name`：批量预测任务的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：存储批量预测结果的云存储位置。
- `sync`：如果设置为 True，则调用会阻塞，直到异步的批量任务完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="biomedical_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    sync=False,
)

print(batch_predict_job)

等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的 Cloud 存储输出桶中。您可以调用 `iter_outputs()` 方法来获取包含结果的每个 Cloud 存储文件的列表。每个文件以 JSON 格式包含一个或多个预测请求：

- `content`: 预测请求。
- `prediction`: 预测响应。
 - `ids`: 每个预测请求的内部分配的唯一标识符。
 - `displayNames`: 每个类别标签的类名。
 - `confidences`: 每个类别标签的预测置信度，介于 0 和 1 之间。
 - `textSegmentStartOffsets`: 文本中实体开始的字符偏移量。
 - `textSegmentEndOffsets`: 文本中实体结束的字符偏移量。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- 定制作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME